In [2]:
from attr_rec import *
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import re

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
att_df = pd.read_json('../data/attraction/attractions.json',orient='records')

## User details and Preferences

In [4]:
options_place = list(att_df.groupby('city').size().reset_index().sort_values([0], ascending=True).city.values)

uname = widgets.Text(description="Tên người dùng", style=dict(description_width='111px'))
place = widgets.Dropdown(options=options_place, description='Điểm đến (Tỉnh/TP)', style=dict(description_width='111px'))
budget = widgets.IntRangeSlider(min=0, max=10000000, step=10000, value=[0, 10000000], description="Kinh phí (VND)", style=dict(description_width='111px'))
v1 = widgets.VBox([uname,place,budget])
v1.layout.width = '400px'
start = widgets.DatePicker(description='Ngày bắt đầu', disabled=False, style=dict(description_width='100px'))
end = widgets.DatePicker(description='Ngày kết thúc', disabled=False, style=dict(description_width='100px'))
v2 = widgets.VBox([start,end])
out = widgets.HBox([v1,v2])
display(out)

In [5]:
category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)

cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = widgets.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = widgets.Layout(width='100%', height='100px')
but_items = [widgets.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = widgets.VBox([i for i in but_items[:4]])
c2 = widgets.VBox([i for i in but_items[4:8]])
c3 = widgets.VBox([i for i in but_items[8:12]])
c4 = widgets.VBox([i for i in but_items[12:16]])
c5 = widgets.VBox([i for i in but_items[16:]])
buttons = widgets.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

Select and rate atleast 5 categories and rate them:


Thủy cung, vui chơi giải trí


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

Chụp ảnh, trải nghiệm, thư giãn


IntSlider(value=0, description='Rate', max=5)

...

IntSlider(value=0, description='Rate', max=5)

Rate 2 more!

Tàu điện, chụp ảnh


IntSlider(value=0, description='Rate', max=5)

Rate 1 more!

Kiến trúc, nghệ thuật


IntSlider(value=0, description='Rate', max=5)

In [6]:
user_name = re.sub(' ','_',uname.value.lower())
city = place.value
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value.value)

In [7]:
cat_rating
low
high

10000000.0

## Model recommendation and filtering

In [23]:
filename, user, rbm_att = get_recc(att_df, cat_rating)
filtered = filter_df(filename, user, low, high, city, att_df)

Reading the data
Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 100
Model restored


,attraction_id,Recommendation Score
0,1,0.538784
1,2,0.535408
2,4,0.518153
3,5,0.525091
4,6,0.536554
...,...,...
177,188,0.537920
178,189,0.524384
179,190,0.539867
180,191,0.518692


,attraction_id,name,category,city,latitude,longitude,avg_price,rating
0,3,STAR3 CLUB,"Vui chơi, giải trí về đêm",Hà Nội,21.046258,105.840704,804894,4.4
1,8,Nhà Thờ Cửa Bắc,"Văn hóa tôn giáo, tâm linh",Hà Nội,21.041275,105.840432,171378,4.6
2,20,Vườn hoa Lênin,"Đi dạo, ngắm cảnh",Hà Nội,21.031572,105.839212,661955,4.3
3,30,Chùa Một Cột,"Văn hóa tôn giáo, tâm linh",Hà Nội,21.035667,105.832563,116520,4.5
4,33,Chè Thái Đội Cấn,"Ẩm thực, nghỉ ngơi",Hà Nội,21.034317,105.829704,152360,4.1
...,...,...,...,...,...,...,...,...
17,19,Bảo tàng Mỹ thuật Việt Nam,"Văn hóa, lịch sử, nghệ thuật",Hà Nội,21.030689,105.837908,261830,4.2
65,70,Cộng Cà Phê Trung Hòa,"Đồ uống, thư giãn",Hà Nội,21.017312,105.798586,825878,4.2
57,62,Soya Garden,"Đồ uống, thư giãn",Hà Nội,21.038046,105.815239,88239,4.3
91,97,Highlands coffee Yên Lãng,"Đồ uống, thư giãn",Hà Nội,21.011339,105.817506,321576,4.2


In [24]:
# %%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['avg_price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+2):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(filtered, final)
    else:
        final = find_closest(filtered, final['location'][-1], final['timeofday'][i], final)


Item no.: 1 --> Item name = STAR3 CLUB
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = Hanoi Studio Gallery
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = Manzi Art Space and Cafe
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = T\u1ed4 CHIM XANH - BLUEBIRDS' NEST
Evaluating...
Starting Download...

Errors: 0



## Final output

In [24]:
days = (end_date - begin_date).days + 1
display(final_output(days,final))

name:
0
['Hồ trúc bạch', 'Hoàng thành thăng long', 'Phố đường tàu hà nội', 'San ho seafood restaurant']


name:
1
['Công viên indira gandhi', 'Tiny cafe', 'Paris gateaux cafe', "Pizza 4p's phan ké bính"]


name:
2
['Gogi house', 'Highlands coffee pullman hn', 'Chùa bộc', 'Vua chả cá mai hắc đế']


name:
3
['The coffee house vũ phạm hàm', 'Starbucks ba trieu', 'Tràng tiền plaza', 'Bún đậu cô tuyến mắm tôm hàng khay']




In [25]:
cat_rating

{'Đồ uống, thư giãn': 4.0,
 'Ẩm thực, nghỉ ngơi': 4.0,
 'Văn hóa, lịch sử, nghệ thuật': 4.0,
 'Văn hóa tôn giáo, tâm linh': 4.0,
 'Công viên, đi dạo, ngắm cảnh': 4.0}